In [1]:
#model轉onnx(C2FViT)
import os
import torch
import torch.onnx
# import voxelmorph with pytorch backend
os.environ['NEURITE_BACKEND'] = 'pytorch'
os.environ['VXM_BACKEND'] = 'pytorch'
import voxelmorph as vxm

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# 使用你的自定義加載方法載入模型
model = vxm.networks.VxmDense.load('/NFS/PeiMao/voxelmorph/model_ABIDE_torch_NCC_LambdaSet1_validation/best.pt', 'cuda')
model.to('cuda')
model.eval()

# 創建一個適當大小的範例輸入
example_input1 = torch.randn(1, 1, 160, 224, 192).to('cuda')
example_input2 = torch.randn(1, 1, 160, 224, 192).to('cuda')

# 轉換模型
torch.onnx.export(model, (example_input1, example_input2), 'mprage_reg_v002_train.onnx', input_names=["src", "trg"], output_names=["moved", "warp"], opset_version=16)

/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/conda/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
#讀取onnx
import onnxruntime as ort

# 假設你的模型文件名為 'model.onnx'
model_path = 'mprage_reg_v002_train.onnx'

# 創建一個 ONNX Runtime 會話
session = ort.InferenceSession(model_path)
print("Model loaded successfully.")

Model loaded successfully.


In [3]:
#讀取資料並預測
import onnxruntime as ort
import numpy as np
import nibabel as nib

moving_nii = nib.load('/NFS/PeiMao/dataset/ABIDE_reshape_test/ABIDE_0050003_tbet.nii.gz')
#fixed_nii = nib.load('/NFS/PeiMao/dataset/ABIDE_reshape_test/ABIDE_0050012_tbet.nii.gz')
fixed_nii = nib.load('/NFS/PeiMao/voxelmorph/mni152_reshape_norm.nii.gz')
fixed_affine = fixed_nii.affine
moving_data = moving_nii.get_fdata().astype(np.float32)
fixed_data = fixed_nii.get_fdata().astype(np.float32)

# 在第0轴和第1軸位置添加新维度（增加一个 batch size 维度）
moving = np.expand_dims(moving_data, axis=0)
moving = np.expand_dims(moving, axis=1)
fixed = np.expand_dims(fixed_data, axis=0)
fixed = np.expand_dims(fixed, axis=1)

# 创建 ONNX Runtime 会话
session = ort.InferenceSession("mprage_reg_v002_train.onnx")

# 获取输入的名称
input_names = [input.name for input in session.get_inputs()]
output_names = [output.name for output in session.get_outputs()]

# 创建输入字典
inputs = {input_names[0]: moving, input_names[1]: fixed}

# 运行模型推理
outputs = session.run(None, inputs)

# 打印输出结果
print(outputs[0].shape)
print(outputs[1].shape)
#print(input_names)
#print(output_names)


# 使用 squeeze 移除长度为 1 的维度
moved = np.squeeze(outputs[0])
print(moved.shape)

warp = np.squeeze(outputs[1])
print(warp.shape)

# 创建一个 NIfTI 图像对象
moved_nii = nib.Nifti1Image(moved, fixed_affine)
warp_nii = nib.Nifti1Image(warp, fixed_affine)

# 保存 NIfTI 图像为 .nii.gz 文件
nib.save(moved_nii, 'onnx_output_image.nii.gz')
nib.save(warp_nii, 'onnx_output_image_warp.nii.gz')

(1, 1, 160, 224, 192)
(1, 3, 160, 224, 192)
(160, 224, 192)
(3, 160, 224, 192)


In [7]:
import torch
import torch.onnx
from C2FViT_model import C2F_ViT_stage
import os

# 加載模型
model = C2F_ViT_stage(img_size=128, patch_size=[3, 7, 15], stride=[2, 4, 8], num_classes=12,
                      embed_dims=[256, 256, 256],
                      num_heads=[2, 2, 2], mlp_ratios=[2, 2, 2], qkv_bias=False, qk_scale=None, drop_rate=0.,
                      attn_drop_rate=0., norm_layer=torch.nn.Identity,
                      depths=[4, 4, 4], sr_ratios=[1, 1, 1], num_stages=3, linear=False).to('cuda')

model.load_state_dict(torch.load('../Model/C2FViT_affine_COM_template_matching_stagelvl3_116000.pth'))
model.eval()

# 建立範例輸入張量
#example_input1 = torch.randn(1, 1, 128, 128, 128).to('cuda')
#example_input2 = torch.randn(1, 1, 128, 128, 128).to('cuda')
example_input1 = torch.randn(1, 1, 256, 256, 256).to('cuda')
example_input2 = torch.randn(1, 1, 256, 256, 256).to('cuda')

# 將模型轉換為ONNX格式
torch.onnx.export(model, 
                  (example_input1, example_input2), 
                  'C2FViT_model.onnx', 
                  input_names=["src", "trg"], 
                  output_names=["moved", "warp"], 
                  opset_version=20)

print("Model has been converted to ONNX.")


/tmp/ipykernel_200887/2136736169.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../Model/C2FViT_affine_COM_template_matching_stagelvl

OutOfMemoryError: CUDA out of memory. Tried to allocate 8.00 GiB. GPU 0 has a total capacity of 47.53 GiB of which 6.69 GiB is free. Process 265369 has 40.82 GiB memory in use. Of the allocated memory 34.50 GiB is allocated by PyTorch, and 6.02 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [17]:
#讀取資料並預測
import onnxruntime as ort
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom

def pad_to_shape(img, target_shape):
    """
    Pads the input image with zeros to match the target shape.
    """
    padding = [(max(0, t - s)) for s, t in zip(img.shape, target_shape)]
    pad_width = [(p // 2, p - (p // 2)) for p in padding]
    padded_img = np.pad(img, pad_width, mode='constant', constant_values=0)
    return padded_img
    
moving_nii = nib.load('/NFS/PeiMao/GitHub/C2FViT_Medical_Image/Data/ABIDE_0050003_tbet_reoriented.nii.gz')
#fixed_nii = nib.load('/NFS/PeiMao/dataset/ABIDE_reshape_test/ABIDE_0050012_tbet.nii.gz')
fixed_nii = nib.load('/NFS/PeiMao/GitHub/C2FViT_Medical_Image/Data/MNI152_T1_1mm_brain_pad_RSP.nii.gz')
fixed_affine = fixed_nii.affine
moving_data = moving_nii.get_fdata().astype(np.float32)
fixed_data = fixed_nii.get_fdata().astype(np.float32)

moving_data = pad_to_shape(moving_data, (256, 256, 256))
moving_data = zoom(moving_data, (0.5, 0.5, 0.5), order=1)  # order=1表示雙線性插值
fixed_data = zoom(fixed_data, (0.5, 0.5, 0.5), order=1)  # order=1表示雙線性插值


# 在第0轴和第1軸位置添加新维度（增加一个 batch size 维度）
moving = np.expand_dims(moving_data, axis=0)
moving = np.expand_dims(moving, axis=1)
fixed = np.expand_dims(fixed_data, axis=0)
fixed = np.expand_dims(fixed, axis=1)

# 创建 ONNX Runtime 会话
session = ort.InferenceSession("C2FViT_model.onnx")

# 获取输入的名称
input_names = [input.name for input in session.get_inputs()]
output_names = [output.name for output in session.get_outputs()]

# 创建输入字典
inputs = {input_names[0]: moving, input_names[1]: fixed}

# 运行模型推理
outputs = session.run(None, inputs)

# 打印输出结果
print(outputs[0].shape)
print(outputs[1].shape)
#print(input_names)
#print(output_names)


# 使用 squeeze 移除长度为 1 的维度
moved = np.squeeze(outputs[0])
print(moved.shape)

warp = np.squeeze(outputs[1])
print(warp.shape)

# 创建一个 NIfTI 图像对象
moved_nii = nib.Nifti1Image(moved, fixed_affine)
warp_nii = nib.Nifti1Image(warp, fixed_affine)

# 保存 NIfTI 图像为 .nii.gz 文件
nib.save(moved_nii, 'onnx_output_image.nii.gz')
nib.save(warp_nii, 'onnx_output_image_warp.nii.gz')

(1, 1, 64, 64, 64)
(1, 1, 128, 128, 128)
(64, 64, 64)
(128, 128, 128)


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import nibabel as nib
import numpy as np
from C2FViT_model import C2F_ViT_stage, AffineCOMTransform, Center_of_mass_initial_pairwise
from Functions import min_max_norm, pad_to_shape

class FullModel(nn.Module):
    def __init__(self, model, affine_transform, init_center):
        super(FullModel, self).__init__()
        self.model = model
        self.affine_transform = affine_transform
        self.init_center = init_center

    def forward(self, moving_img, fixed_img):
        # Center of mass initialization
        moving_img, init_flow = self.init_center(moving_img, fixed_img)
        
        # Downsample the images
        X_down = F.interpolate(moving_img, scale_factor=0.5, mode="trilinear", align_corners=True)
        Y_down = F.interpolate(fixed_img, scale_factor=0.5, mode="trilinear", align_corners=True)
        
        # Run the core model
        warpped_x_list, y_list, affine_para_list = self.model(X_down, Y_down)
        
        # Apply the affine transformation
        X_Y, affine_matrix = self.affine_transform(moving_img, affine_para_list[-1])
        
        return X_Y, affine_matrix

# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義模型
model = C2F_ViT_stage(img_size=128, patch_size=[3, 7, 15], stride=[2, 4, 8], num_classes=12,
                      embed_dims=[256, 256, 256], num_heads=[2, 2, 2], mlp_ratios=[2, 2, 2], qkv_bias=False,
                      qk_scale=None, drop_rate=0., attn_drop_rate=0., norm_layer=nn.Identity,
                      depths=[4, 4, 4], sr_ratios=[1, 1, 1], num_stages=3, linear=False).to(device)

# 加載預訓練模型權重
model_path = '../Model/C2FViT_affine_COM_template_matching_tigerdata_RAS/C2FViT_affine_COM_template_matching_tigerdata_RAS_stagelvl3_27000.pth'
print(f"Loading model weight {model_path} ...")
model.load_state_dict(torch.load(model_path))
model.eval()

# 定義轉換器
affine_transform = AffineCOMTransform().to(device)
init_center = Center_of_mass_initial_pairwise()

# 將核心模型和轉換器打包成完整模型
full_model = FullModel(model, affine_transform, init_center).to(device)

# 加載固定影像
fixed_path = '../Data/MNI152_T1_1mm_brain_pad_RSP.nii.gz'
fixed_img_nii = nib.load(fixed_path)
fixed_img = fixed_img_nii.get_fdata()

# 確保影像尺寸是 256x256x256
target_shape = (256, 256, 256)
if fixed_img.shape != target_shape:
    fixed_img = pad_to_shape(fixed_img, target_shape)

fixed_img = min_max_norm(fixed_img)
fixed_img = torch.from_numpy(fixed_img).float().to(device).unsqueeze(0).unsqueeze(0)

# Dummy moving image for ONNX conversion
dummy_moving_img = torch.randn(1, 1, 256, 256, 256).to(device)

# 將完整模型轉換成 ONNX 格式
onnx_path = "C2FViT_full_model.onnx"
torch.onnx.export(full_model, 
                  (dummy_moving_img, fixed_img), 
                  onnx_path, 
                  export_params=True, 
                  opset_version=20, 
                  do_constant_folding=True, 
                  input_names=['moving_img', 'fixed_img'], 
                  output_names=['moved', 'affine_matrix'])

print(f"Full model has been converted to {onnx_path}")


/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Loading model weight ../Model/C2FViT_affine_COM_template_matching_tigerdata_RAS/C2FViT_affine_COM_template_matching_tigerdata_RAS_stagelvl3_27000.pth ...


/tmp/ipykernel_204070/4042287221.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Full model has been converted to C2FViT_full_model.onnx


In [13]:
#讀取資料並預測
import onnxruntime as ort
import numpy as np
import nibabel as nib
from Functions import min_max_norm, pad_to_shape, reorient_image

    
moving_nii = nib.load('/NFS/PeiMao/GitHub/C2FViT_Medical_Image/Data/ABIDE_0050003_tbet.nii.gz')
fixed_nii = nib.load('/NFS/PeiMao/GitHub/C2FViT_Medical_Image/Data/MNI152_T1_1mm_brain_pad_RSP_RAS.nii.gz')
fixed_affine = fixed_nii.affine
fixed_header = fixed_nii.header
moving_nii = reorient_image(moving_nii, ('R', 'A', 'S'))
moving_data = moving_nii.get_fdata().astype(np.float32)
fixed_data = fixed_nii.get_fdata().astype(np.float32)
moving_data = pad_to_shape(moving_data, (256, 256, 256))

fixed_data = np.clip(fixed_data, a_min=2500, a_max=np.max(fixed_data))

# 在第0轴和第1軸位置添加新维度（增加一个 batch size 维度）
moving = np.expand_dims(moving_data, axis=0)
moving = np.expand_dims(moving, axis=1)
fixed = np.expand_dims(fixed_data, axis=0)
fixed = np.expand_dims(fixed, axis=1)

moving = min_max_norm(moving)
fixed = min_max_norm(fixed)

# 创建 ONNX Runtime 会话
session = ort.InferenceSession("C2FViT_full_model.onnx")

# 获取输入的名称
input_names = [input.name for input in session.get_inputs()]
output_names = [output.name for output in session.get_outputs()]

# 创建输入字典
inputs = {input_names[0]: moving, input_names[1]: fixed}

# 运行模型推理
outputs = session.run(None, inputs)

# 打印输出结果
print(outputs[0].shape)
print(outputs[1].shape)

# 使用 squeeze 移除长度为 1 的维度
moved = np.squeeze(outputs[0])
affine_matrix = np.squeeze(outputs[1])
print(affine_matrix)

# 创建一个 NIfTI 图像对象
moved_nii = nib.Nifti1Image(moved, fixed_affine)

# 保存 NIfTI 图像为 .nii.gz 文件
nib.save(moved_nii, 'onnx_output_image.nii.gz')

(1, 1, 256, 256, 256)
(1, 3, 4)
[[ 0.80465275  0.37881488 -0.02458961  0.00602981]
 [-0.34743226  0.8497192   0.03234605 -0.02687525]
 [ 0.03633387 -0.01498665  0.8969442   0.0021124 ]]
